In [1]:
%display latex

%runfile diagonal_polynomial_ring.py
%runfile subspace.py
%runfile young_idempotent.py
%runfile add_degree.py
%runfile diagram.py

SymmetricFunctions(QQ).inject_shorthands(verbose=False)

#%runfile compute_character.py

# Code

Calcul du Vandermonde associé au diagramme $\gamma$, où $\gamma$ est un partage (Partition) ou un diagramme (Diagram).

In [2]:
def vandermonde(gamma, r=0):
    n = gamma.size()
    if r == 0:
        r = 1
    P = DiagonalPolynomialRing(QQ, n, r, inert=1) 
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

Fonctions qui nous renvoient des listes d'opérateurs par degrés pour toutes les variables (non inertes) de l'anneau de polynômes concerné.

In [3]:
def partial_derivatives(P):
    """
    Return the partial derivative functions in all the variables of `P`.
    
    INPUT:
    - `P` -- a diagonal polynomial ring
    
    """
    n = P.ncols()
    r = P.nrows()
    D = P.grading_set()
    X = P.derivative_variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [attrcall("derivative", X[i,k]) for k in range(n)]
    return op

def polarization_operators(r, max_deg=1, side=None, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    return {D([-d if i==i1 else 1 if i==i2 else 0 for i in range(r)]):
            [attrcall("polarization", i1=i1, i2=i2, d=d, row_symmetry=row_symmetry)]
            for d in range(1, max_deg+1)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

##############################################################

def steenrod_operators(r, degree=1, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [
            attrcall("steenrod_op", i=i, k=degree+1, row_symmetry=row_symmetry)]
    return op

def symmetric_derivatives(r, list_deg, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    return {D(-i for i in d) : [attrcall("symmetric_derivative", d=d, row_symmetry=row_symmetry)] 
            for d in list_deg}

In [4]:
def merge(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

def antisymmetries(mu):
    mu = Partition(mu)
    return antisymmetries_of_tableau(mu.initial_tableau())

Projection sur les composantes isotypiques; prend en paramètre un sous-espace (Subspace) et un entier ou un partage. 

In [5]:
def IsotypicComponent(S, arg, use_antisymmetry=False):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    P1 = basis.values().pop()[0].parent()
    for nu in list_partitions:
        result_nu = {}
        if use_antisymmetry == True:
            antisymmetries = antisymmetries_of_tableau(nu.initial_tableau())
            P2 = DiagonalAntisymmetricPolynomialRing(P1._R, P1.ncols(), P1.nrows(), 
                                                 P1.ninert(), antisymmetries)
        for deg, value in basis.iteritems():
            if use_antisymmetry:
                gen = []
                for p in value:
                    temp = apply_young_idempotent(P2(p), nu)
                    if temp != 0: 
                        gen += [temp]
            else:
                gen = []
                for p in value:
                    temp = apply_young_idempotent(p, nu)
                    if temp != 0:
                        gen += [temp]
            if gen != []:
                result_nu[(deg, tuple(nu))] = Subspace(gen, {}).basis()[0]
        if result_nu != {}:
            result[nu] = Subspace(result_nu, operators={})
                
    if len(result.keys()) == 1:
        key = result.keys()[0]
        return result[key]
    else:
        return result

In [6]:
def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]

def add_degrees_symmetric(gen_deg,op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    d = D(gen_deg[0])+D(op_deg)
    return D(sorted(d, reverse=True)), gen_deg[1]

Création de l'espace polarisé par les opérateurs donnés en paramètres.

In [7]:
def PolarizedSpace(S, operators, add_degrees=add_degrees_isotypic):
    if isinstance(S, dict):
        return {key : PolarizedSpace(value, operators, add_degrees=add_degrees)
                for key, value in S.iteritems()}
    else:
        basis = S.basis()
        basis_element = basis.values().pop()[0]
        P1 = basis_element.parent()
        r = len(op_pol.keys().pop())
        row_symmetry = op_pol.values().pop()[0].kwds['row_symmetry']
        if row_symmetry == "permutation":
            add_degrees = add_degrees_symmetric
        D = cartesian_product([ZZ for i in range(r)])
        generators = {}

        if isinstance(P1, DiagonalAntisymmetricPolynomialRing):
            P2 = DiagonalAntisymmetricPolynomialRing(P1._R, P1.ncols(), r , P1.ninert(), P1.antisymmetries())
            for key, value in basis.iteritems():
                d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
                generators[d] = tuple(reduce_antisymmetric_normal(P2(b), 
                                                      b.parent().ncols(), 
                                                      b.parent().nrows()+b.parent().ninert(), 
                                                      b.antisymmetries()) for b in value)
        else :
            P2 = DiagonalPolynomialRing(P1._R, P1.ncols(), r , P1.ninert())
            for key, value in basis.iteritems():
                d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
                generators[d] = tuple(P2(b) for b in value)
        return Subspace(generators, operators, add_degrees=add_degrees)

Calcul de l'image de S (Subspace) par les opérateurs donnés.

In [8]:
def Range(S, operators, add_degrees=add_degrees_isotypic):
    if isinstance(S, dict):
        return {key : Range(value, operators, add_degrees=add_degrees)
                for key, value in S.iteritems()}

    result = {}
    basis = S.basis()
    for key, b in basis.iteritems():
        result = merge(result, {add_degrees(key, deg): 
                                     [op(p) for p in b for op in op_list if op(p)!=0] 
                                     for deg, op_list in operators.iteritems()})    
    if result != {} :
        return Subspace(result, {}, add_degrees) #{} <-> operators
    else :
        return None

Calcul du caractère à partir d'un sous-espace (S) d'éléments projetés sur les composantes isotypiques.

In [9]:
def character(S, left_basis=s, right_basis=s, row_symmetry=None):
    if isinstance(S, dict):
        return sum(character(V,
                             left_basis=left_basis, right_basis=right_basis, row_symmetry=row_symmetry) 
                   for V in S.values())
    else:
        basis = S.basis()
        basis_element = basis.values().pop()[0]
        P = basis_element.parent()
        n = P.ncols()
        r = P.nrows()
        
        charac = 0
        if row_symmetry != "permutation":
            q = PolynomialRing(QQ,'q',r).gens()

        for nu in Partitions(n):
            basis_nu = {}
            charac_nu = 0
            # Get the nu_isotypic part of the basis
            for key, value in basis.iteritems():
                if Partition(key[1]) == nu:
                    basis_nu[key[0]] = value

            # Use monomials to compute the character
            if row_symmetry == "permutation":
                for deg, b in basis_nu.iteritems():
                    charac_nu += sum(m(Partition(deg)) for p in b)
                if charac_nu != 0 :
                    if left_basis == s :
                        charac_nu = s(charac_nu).restrict_partition_lengths(r,exact=False)
                    elif left_basis != m :
                        charac_nu = left_basis(charac_nu)

            # Or use directly the degrees
            else:
                for deg, b in basis_nu.iteritems():
                    charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
                if charac_nu != 0 :
                    if left_basis == s :
                        charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
                    else:
                        charac_nu = left_basis.from_polynomial(charac_nu)

            # Make the tensor product with s[nu]
            if charac_nu != 0:
                charac += tensor([charac_nu, right_basis(s(nu))])
        return charac

Factorisation du produit tensoriel par représentation de $S_n$. Et calcul de la dimension.

In [10]:
def factorise(f, n):
    SymmetricFunctions(QQ).s()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu])) for mu in result.keys()]
    for a, b in result2:
        if b!=0:
            print a
            show(b)
        
def dimension(f, n):
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(1, alphabet=['q'])) for mu in result.keys()]
    q = result2[0][1].parent().gens()[0]
    return [(tuple(a), b.subs({q:1})) for a,b in result2]

In [27]:
def compute_character(mu, use_antisymmetry=True, row_symmetry="permutation"):
    n = Integer(mu.size())
    # Determinant computation
    v = vandermonde(mu)
    # Span by derivatives
    generator = {v.multidegree() : [v]}
    list_op = partial_derivatives(v.parent())
    V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
    # Projection on isotypic components
    V_iso = IsotypicComponent(V, n, use_antisymmetry=use_antisymmetry)
    # Polarization
    r = n-1
    deg = v.degree()
    if deg == 0:
        deg = 1
    op_pol = polarization_operators(r, deg, row_symmetry=row_symmetry)
    V_pol = PolarizedSpace(V_iso, op_pol)
    
    # character
    return character(V_pol, row_symmetry=row_symmetry)

# Exemples de calculs

### Cas du diagramme 013

In [12]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

On crée le sous-espace engendré par $\mathcal{V}_\gamma$ et ses dérivées partielles. On peut ajouter les opérateurs de Steenrod sans changement du résultat.

In [13]:
generator = {v.multidegree() : [v]}
list_op = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))

In [14]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00^2 - x02^2, x01^2 - x02^2, -x00*x01 + x00*x02, -x00*x01 + x01*x02),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3),
 (4,): (-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3,)}

In [15]:
character(IsotypicComponent(W1, 3))

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Appliquons $\sum_i \partial_{x_i}$ et $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [16]:
v.symmetric_derivative((1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [17]:
v.steenrod_op(0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarisation

In [18]:
op_pol = polarization_operators(2)
W2 = PolarizedSpace(IsotypicComponent(W1, 3), op_pol)
character(W2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [19]:
r=2
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(r)) 
                for k1 in range(1, 3+1) for i1 in range(0, r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(r, list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(r, list_degrees), 
#                       steenrod_operators(r, 1)), steenrod_operators(r, 2))
character(Range(W2, sym_diff))

s[] # s[3] + s[1] # s[2, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [20]:
charac = character(W2) - character(Range(W2, sym_diff))
charac

s[1] # s[3] + s[2] # s[2, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Ou dans la base des monomiales

In [21]:
character(W2, left_basis=m) - character(Range(W2, sym_diff), left_basis=m)

m[1] # s[3] + m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [22]:
factorise(charac, 3)

[1, 1, 1]


s[4]

[3]


s[1]

[2, 1]


s[2] + s[3]

### Autres cas

Pas de quotient dans ce cas.

In [28]:
compute_character(Partition([2]))

s[] # s[2] + s[1] # s[1, 1]

In [29]:
compute_character(Partition([3,1]))

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] - s[1, 1] # s[2, 1, 1] + s[2] # s[2, 1, 1] - s[2, 1] # s[1, 1, 1, 1] + s[3] # s[1, 1, 1, 1]

In [30]:
compute_character(Partition([1,1,1]))

s[] # s[1, 1, 1]

In [31]:
compute_character(Partition([2,1,1]))

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]